In [ ]:
# v2: 
# import of PET
# alignment of CT and BBprediction with PET
# PET conversion to SUVbw

# v2.1
# uses latest BB prediction model:
#"30ptsChris_3D_ExtendedSlicesWithBB_Unet_3level_5conv_16slice_BCE_LRe-4_systematic_order_500_ValLoss_best_mdl_wts"
# extracts SUV data from BB prediction

#v2.2
# incorporates largest 3D connected component

#v2.3_GUIprep
# incorporates removal of peripheral PET uptake that amounts to bladder/kidneys/liver etc. 
# started gathering some stats at the end (min, max, mean, percentiles etc).

# v2.3_GUI_v1.0
# actually made to display a GUI window


In [ ]:
import tkinter as tk
from tkinter import *

In [ ]:
from pandas import DataFrame
from matplotlib import pyplot as plt, cm
from matplotlib import image
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [ ]:
import math
import sys
import json
import dicom
import dicom_numpy
import os
import numpy as np
from numpy import flip
import nibabel as nib
import math
from scipy.ndimage import zoom
import datetime
from scipy.ndimage.measurements import label

In [ ]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

In [ ]:
from keras.models import Model, model_from_json, load_model
# from keras.layers import Input, Dropout, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose
from keras.layers import Input, Dropout, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, concatenate, Conv2DTranspose, Conv3DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras import losses
K.set_image_dim_ordering('th')  # Theano dimension ordering in this code. Would need "tensorflow" ordering for CPU. 

In [ ]:
sys.executable

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

## Import and prep CTpet and PET data


In [ ]:
# #user defines location of the CT DICOM files
# baseCT = "D:/CNNdata/Bowel_Segmentation/RATHL_Data_Validation/CT"
# basePET = "D:/CNNdata/Bowel_Segmentation/RATHL_Data_Validation/PT"
# ID = 1

# baseCT = "D:/CNNdata/Bowel_Segmentation/RATHL_Data_Validation_Extra/CT"
# basePET = "D:/CNNdata/Bowel_Segmentation/RATHL_Data_Validation_Extra/PT"
# ID = "_RATHL02_1014"
## User will define the sup and inf limits of bowelbag prediction - probably from GUI
## as slice number from inf end in normal numbering i.e. from 1
## and the slice position then calculated later from b_CT
# BBinfSliceNo = 62 # in normal numbering i.e. from 1
# BBsupSliceNo = 157 # in normal numbering i.e. from 1



In [ ]:


# function to extract voxel data from DICOM to Numpy using the dicom_numpy module
# seems to take into account the rescale intercept appropriately

def extract_voxel_data(list_of_dicom_files):
    datasets = [dicom.read_file(f) for f in list_of_dicom_files]
    try:
        voxel_ndarray, ijk_to_xyz = dicom_numpy.combine_slices(datasets)
    except dicom_numpy.DicomImportException as e:
        # invalid DICOM data
        raise
    return voxel_ndarray

## create the list of DICOM files in the CT and PET folders

def listofDICOMfiles():
    
    global lstFilesDCM_CT
    global lstFilesDCM_PET
    
    PathPETCT = ImportPETCT_txt.get()
    print("PathPETCT = ",PathPETCT)

    PatientID = PatientID_txt.get()
    print("PatientID = ",PatientID)

    PathDicomCT = os.path.join(str(PathPETCT) + "/CT" + str(PatientID))
    PathDicomPET = os.path.join(str(PathPETCT) + "/PT" + str(PatientID))

    lstFilesDCM_CT = []  # create an empty list
    for dirName, subdirList, fileList in os.walk(PathDicomCT):
        for filename in fileList:
            if ".dcm" in filename.lower():  # check whether the file's DICOM
                lstFilesDCM_CT.append(os.path.join(dirName,filename))

    lstFilesDCM_PET = []  # create an empty list
    for dirName, subdirList, fileList in os.walk(PathDicomPET):
        for filename in fileList:
            if ".dcm" in filename.lower():  # check whether the file's DICOM
                lstFilesDCM_PET.append(os.path.join(dirName,filename)) 

def ReadDicoms():
    
    global b_CT
    global CTarray
    global PETarray
    global PETReconDiameter
    global CTReconDiameter
    global ConstPixelDims_CT
    global ConstPixelDims_PET
    
    # Get ref file
    RefCT = dicom.read_file(lstFilesDCM_CT[0])
    # Load dimensions based on the number of rows, columns, and slices (along the Z axis)
    ConstPixelDims_CT = (int(RefCT.Rows), int(RefCT.Columns), len(lstFilesDCM_CT))
    # Load spacing values (in mm)
    ConstPixelSpacing_CT = (float(RefCT.PixelSpacing[0]), float(RefCT.PixelSpacing[1]), float(RefCT.SliceThickness))
    CTReconDiameter = round((RefCT.PixelSpacing[0]) * (RefCT.Rows))
    print("CTReconDiameter = ", CTReconDiameter)

    # Get header info from PET
    RefPET = dicom.read_file(lstFilesDCM_PET[0])
    # Load dimensions based on the number of rows, columns, and slices (along the Z axis)
    ConstPixelDims_PET = (int(RefPET.Rows), int(RefPET.Columns), len(lstFilesDCM_PET))
    # Load spacing values (in mm)
    ConstPixelSpacing_PET = (float(RefPET.PixelSpacing[0]), float(RefPET.PixelSpacing[1]), float(RefPET.SliceThickness))

    PETReconDiameter = round((RefPET.PixelSpacing[0]) * (RefPET.Rows))
    print("PETReconDiameter = ", PETReconDiameter)
    #PETReconDiameter
    
    # This is needed later to find the slice number of the first slice of BB
    # The array is sized based on 'ConstPixelDims'
    a_CT = np.zeros(ConstPixelDims_CT[2])
    i=0
    # loop through all the DICOM files, reading the slice numbers to array
    for filenameDCM in lstFilesDCM_CT:
        dsCT = dicom.read_file(filenameDCM)
        sliceNumber = dsCT.ImagePositionPatient[2]
        a_CT[i] = sliceNumber
        i=i+1

    b_CT = np.sort(a_CT, axis=-1, kind='quicksort', order=None) #element 0 is inf-most slice
    #b_CT


    # RefCT.SpacingBetweenSlices is what is needed for slice separation
    # but for some header files it doesn't appear.
    # Therefore backup is to calculate difference between slice positions

    try:
        d = float(RefCT.SpacingBetweenSlices)

    except AttributeError:
        # Creating the Big Object takes five days
        # and three hundred pounds of over-ripe melons.
        d = round(b_CT[1] - b_CT[0],2)

    print(d)
    
    CTarray = extract_voxel_data(lstFilesDCM_CT)
    PETarray = extract_voxel_data(lstFilesDCM_PET)


In [ ]:


def ArrayMods():
    
    global CTarray
    global PETarray
    global TestSlices
    global CTarrayNormBBslices
    global PETarrayNew
    global BBinfSliceNo
    global BBsupSliceNo
    global Slices
    
    BBinfSliceNo = int(InfSlice_txt.get()) # in normal numbering i.e. from 1
    BBsupSliceNo = int(SupSlice_txt.get()) # in normal numbering i.e. from 1
    
    ## slice position calculated from b_CT
    BBinfSlicePos = b_CT[BBinfSliceNo-1]
    print(BBinfSlicePos)
    BBsupSlicePos = b_CT[BBsupSliceNo-1]
    print(BBsupSlicePos)

    print("CT array shape = ",np.shape(CTarray))
    print("CT minimum HU = ", np.min(CTarray))

    #some CT data has a ring of -3000 values around the FOV circle ... not sure why
    CTarray[CTarray < -1000] = -1000

    print("CT array shape = ",np.shape(CTarray))


    CTarray = np.transpose(CTarray, (2,0,1)) 
    CTarray = np.expand_dims(CTarray, axis=0)
    CTarray = np.expand_dims(CTarray, axis=0)

    print("CT array shape = ",np.shape(CTarray))

    CTarray = np.rot90(CTarray,1,(3,4))
    # for i in range(0,ConstPixelDims_CT[2]):
    #     CTarrayNorm[i,0,:,:] = np.rot90(CTarrayNorm[i,0,:,:],1)

    print("Replace all elements of CT array with 2000 which are greater than 2000HU")
    print("This will set all of the metal to to bone density")
    print("Could set it to air or tissue instead I guess but hip implants should be set to bone at least")
    print("Performed normalisation")

    HighDens = 2000

    CTarray[CTarray > HighDens] = HighDens
    CTarrayNorm=(CTarray/(HighDens+1000)) +(1/3)

    print("PET array shape = ",np.shape(PETarray))

    PETarrayNew = np.transpose(PETarray, (2,0,1)) #numbers indicate where the original dimension should go
    PETarrayNew = np.expand_dims(PETarrayNew, axis = 0) # adds a dimension
    PETarrayNew = np.expand_dims(PETarrayNew, axis = 0) # adds a dimension

    PETarrayNew = np.rot90(PETarrayNew,1,(3,4))


    print("CT array shape =  ", np.shape(CTarrayNorm))
    print("PET array shape = ",np.shape(PETarrayNew))

    Slices = np.size(CTarrayNorm,2)
    Slices

    CTarrayNormBBslices = CTarrayNorm[:,:,(BBinfSliceNo-1):(BBsupSliceNo),:,:]
    TestSlices = np.size(CTarrayNormBBslices,2)
    TestSlices

    print("CTarray BBslices shape = ",np.shape(CTarrayNormBBslices))
    
    Message_label = Label(window, text = "Array Mods - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

## Tensorflow setup and model definition


In [ ]:

def get_unet_3D_3level():  
    
    #inputs = Input((1, image_rows, image_cols, image_slices))
    inputs = Input((1, image_slices, image_rows, image_cols))
    #inputs = Input((image_slices, image_rows, image_cols))
    
    print(np.shape(inputs))
    conv1 = Conv3D(32, (5, 5, 5), activation='relu', padding='same')(inputs)
    print(np.shape(conv1))
    D1=Dropout(0.2)(conv1) #need this?
    print(np.shape(D1))
    conv1 = Conv3D(32, (5, 5, 5), activation='relu', padding='same')(D1)
    print(np.shape(conv1))
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    print(np.shape(pool1))
    
#     now at 256x256
    
    conv2 = Conv3D(64, (5, 5, 5), activation='relu', padding='same')(pool1)
    print(np.shape(conv2))
    D2=Dropout(0.2)(conv2)
    print(np.shape(D2))
    conv2 = Conv3D(64, (5, 5, 5), activation='relu', padding='same')(D2)
    print(np.shape(conv2))
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    print(np.shape(pool2))
    
#     now at 128x128

    conv3 = Conv3D(128, (5, 5, 5), activation='relu', padding='same')(pool2)
    print(np.shape(conv3))
    D3=Dropout(0.2)(conv3)
    print(np.shape(D3))
    conv3 = Conv3D(128, (5, 5, 5), activation='relu', padding='same')(D3)
    print(np.shape(conv3))
    
    up9 = concatenate([Conv3DTranspose(64, (2, 2, 2), strides=2, padding='valid')(conv3), conv2], axis=1)
    print(np.shape(up9))
    conv9 = Conv3D(64, (5, 5, 5), activation='relu', padding='same')(up9)
    print(np.shape(conv9))
    D9=Dropout(0.2)(conv9)
    print(np.shape(D9))
    conv9 = Conv3D(64, (5, 5, 5), activation='relu', padding='same')(D9)
    print(np.shape(conv9))

    up10 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=2, padding='same')(conv9), conv1], axis=1)
    print(np.shape(up10))
    conv10 = Conv3D(32, (5, 5, 5), activation='relu', padding='same')(up10)
    print(np.shape(conv10))
    D10=Dropout(0.2)(conv10)
    print(np.shape(D10))
    conv10 = Conv3D(32, (5, 5, 5), activation='relu', padding='same')(D10)
    print(np.shape(conv10))

    conv11 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv10) 
    print(np.shape(conv11))

    model = Model(input=inputs, output=conv11)
    
    return model

# 3D 3-level Unet - Auto-segment bowelbag on CTpet


In [ ]:

def loadmodel():

    global image_rows
    global image_cols
    global image_slices
    global TestSlices
    global BatchesPerPt
    global batch_size
    global model
    
    #user needs to set this correctly according to the network's design
    batch_size = 16

    image_rows = 512
    image_cols = 512
    image_slices = batch_size

    model = get_unet_3D_3level()

    for i in range (0,TestSlices):
        BatchesPerPt = math.ceil(TestSlices / batch_size) # rounds up to work out how many batches from the pt
    print("batches per patient=", BatchesPerPt)

    # user to enter location of the model
    ModelLocation = 'D:/CNNdata/Bowel_Segmentation/Model_Latest/'

    # user to enter the name of the model
    ModelName = '30ptsChris_3D_ExtendedSlicesWithBB_Unet_3level_5conv_16slice_BCE_LRe-4_systematic_order_500_ValLoss_best_mdl_wts.hdf5'

    model.load_weights(os.path.join(str(ModelLocation) + str(ModelName)))






In [ ]:
def predictBB():

    global BatchesPerPt
    global BBpredict

    Message_label = Label(window, text = ".......Bowel Space Prediction.......",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)
                
    BBpredict = np.zeros((1,1,TestSlices,512,512), dtype = 'float32')

    batch_end_slice = TestSlices -1  # where numslices is an array of number of slices for each patient

    for j in range (0,BatchesPerPt):

        if j == BatchesPerPt-1: #i.e. if the last batch in that patient, use first "batch_size" slices

            print("batch = ", j)
            batch_end_slice = batch_size - 1
            batch_start_slice = 0
            print("start slice = ", batch_start_slice)
            print("end slice = ", batch_end_slice)
            print("end slice + 1 = ", batch_end_slice + 1)
            BBpredict[0:1,:,batch_start_slice:batch_end_slice+1,:,:] = model.predict(CTarrayNormBBslices[0:1,:,batch_start_slice:batch_end_slice+1,:,:])

        else:
            print("batch = ", j)
            batch_end_slice = batch_end_slice
            batch_start_slice = batch_end_slice - batch_size +1
            print("start slice = ", batch_start_slice)
            print("end slice = ", batch_end_slice)
            print("end slice + 1 = ", batch_end_slice + 1)
            BBpredict[0:1,:,batch_start_slice:batch_end_slice+1,:,:] = model.predict(CTarrayNormBBslices[0:1,:,batch_start_slice:batch_end_slice+1,:,:])
            batch_end_slice = batch_end_slice - batch_size # ready for the next batch

        print("")
        
    Message_label = Label(window, text = "Bowel Space Prediction - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

# Largest component


In [ ]:
def largestComponent():
    
    global BBpredictLabelled
    
    BBpredict3D = BBpredict[0,0,:,:,:]
    np.putmask(BBpredict3D,BBpredict3D >= 0.5, 1)
    np.putmask(BBpredict3D,BBpredict3D < 1, 0)
    structure3D = np.ones((3, 3, 3), dtype=np.int) # a kind of filter applied to get the components
    BBpredictLabelled, ncomponents = label(BBpredict3D, structure3D)
    
    Array_Components = np.zeros((2,ncomponents), dtype = int)
    for component in range (1,ncomponents+1):
        Array_Components[0,component -1] = component
        Array_Components[1,component -1] = np.count_nonzero(BBpredictLabelled == component)
    print(Array_Components)
    
    locmax = np.argmax(Array_Components, axis=1, out=None)
    
    BBpredictLabelled[BBpredictLabelled != Array_Components[0,locmax[1]]] = 0 # i.e. removes all other components
    BBpredictLabelled[BBpredictLabelled > 0 ] = 1 # resets BB largest component to 1

# BB to match CT

In [ ]:

def BBtoMatchCT():

    global CTarrayFull
    global BBpredictFull
    global PETarrayFull

    BBpredictFull = np.zeros((Slices, 512,512), dtype = int)
    CTarrayFull = CTarray[0,0,:,:,:]
    PETarrayFull = PETarrayNew[0,0,:,:,:]

    BBpredictFull = np.zeros((Slices, 512,512), dtype = int)
    
    BBpredictFull[(BBinfSliceNo-1):(BBsupSliceNo),:,:] = BBpredictLabelled
    
    

# Resizing CT to match PET dimensions


In [ ]:
def resampleCT():
    global CTarrayNewResize
    global BBarrayNewResize
    global BBarrayNewResizeOrig
    
    padextra = round((round((PETReconDiameter/CTReconDiameter)*ConstPixelDims_CT[0]) - ConstPixelDims_CT[0])/2)
    padextra
    
    if padextra > 0:
        CTarrayPad = np.pad(CTarrayFull, ((0,0),(padextra,padextra),(padextra,padextra)), mode = 'constant', constant_values = (-1000))
        BBarrayPad = np.pad(BBpredictFull, ((0,0),(padextra,padextra),(padextra,padextra)), mode = 'constant', constant_values = (0))
        print("padextra > 0")

    elif padextra == 0:
        CTarrayPad = np.copy(CTarrayFull)
        BBarrayPad = np.copy(BBpredictFull)
        print("padextra = 0")

    else:
        CTarrayPad = CTarrayFull[:, -padextra:padextra, -padextra:padextra]
        BBarrayPad = BBpredictFull[:, -padextra:padextra, -padextra:padextra]
        print("padextra < 0")
        
    xyZoomFactor = (ConstPixelDims_PET[0]/ConstPixelDims_CT[0])*(CTReconDiameter/PETReconDiameter)
    zZoomFactor = np.size(PETarray,2)/np.size(CTarray,2)

    CTarrayNewResize = zoom(CTarrayPad, (zZoomFactor,xyZoomFactor, xyZoomFactor))
    BBarrayNewResize = zoom(BBarrayPad, (zZoomFactor,xyZoomFactor, xyZoomFactor))
    
    BBarrayNewResizeOrig = np.copy(BBarrayNewResize)
    
    Message_label = Label(window, text = "Resampling - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

In [ ]:
def PETinSUVunits():
    global PETarrayNewSUVbw
    
    for filenameDCM in lstFilesDCM_PET:
        dsPET = dicom.read_file(filenameDCM)
        ds_Units = dsPET[0x0054,0x1001].value
        ds_halfLife = dsPET[0x0054,0x0016][0][0x0018,0x1075].value 
        ds_PtWeight = dsPET.PatientWeight
        ds_PtName = dsPET[0x10,0x10].value
        ds_injectedDose = dsPET[0x0054,0x0016][0][0x0018,0x1074].value
        ds_seriesDate = dsPET[0x0008,0x0021].value
        ds_seriesTime = dsPET[0x0008,0x0031].value
        ds_acquisitionDate = dsPET[0x0008,0x0022].value
        ds_acquisitionTime = dsPET[0x0008,0x0032].value
        ds_scanTime = dsPET[0x0008,0x0031].value
        ds_startTime = dsPET[0x0054,0x0016][0][0x0018,0x1072].value
        ds_rescaleSlope = dsPET[0x0028,0x1053].value
        ds_rescaleIntercept = dsPET[0x0028,0x1052].value
    
    
    try:
        seriesTimeFormat  = datetime.datetime.strptime(ds_seriesTime, "%H%M%S")
    except:
        seriesTimeFormat  = datetime.datetime.strptime(ds_seriesTime, "%H%M%S.%f")

    try:
        acquisitionTimeFormat  = datetime.datetime.strptime(ds_acquisitionTime, "%H%M%S")
    except:
        acquisitionTimeFormat  = datetime.datetime.strptime(ds_acquisitionTime, "%H%M%S.%f")
        
    try:
        startTimeFormat  = datetime.datetime.strptime(ds_startTime, "%H%M%S") # if the time is like this 105600
    except:
        startTimeFormat  = datetime.datetime.strptime(ds_startTime, "%H%M%S.%f") # if the time is decimal like this 105600.00

    try:
        scanTimeFormat  = datetime.datetime.strptime(ds_scanTime, "%H%M%S")
    except:
        scanTimeFormat  = datetime.datetime.strptime(ds_scanTime, "%H%M%S.%f")


    decaytime = (scanTimeFormat - startTimeFormat).total_seconds()
    
    DecayedDose = ds_injectedDose * pow(2,(-decaytime/ds_halfLife))
    
    SUVbwScaleFactor = (ds_PtWeight * 1000) / DecayedDose
    
    PETarrayNewSUVbw = ((PETarrayFull) + ds_rescaleIntercept) * SUVbwScaleFactor

    print(np.shape(CTarrayNewResize))
    print(np.shape(PETarrayNewSUVbw))
    print(np.shape(BBarrayNewResize))

    Message_label = Label(window, text = "Conversion to SUV - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

# Peripheral PET removal


In [ ]:
## here we choose a value of SUV that we want to threshold
## We then remove thresholded regions outside, and on periphery of, the bowel segmentation

def PeripheralPETremoval():
    global BBarrayNewResize
    
    
    SUVthreshold = 2.5 #the SUV threshold that we choose to indicate non-bowel peripheral regions
    PETthresh = np.copy(PETarrayNewSUVbw) # just reassigning the PET array to PETthresh
    np.putmask(PETthresh,PETthresh < SUVthreshold, 0)
    np.putmask(PETthresh,PETthresh >=  SUVthreshold, 1) # PETthresh is now binary map of regions > SUVthreshold
    structurePET3D = np.ones((3, 3, 3), dtype=np.int) # a kind of filter applied to get the components
    PETLabelled, nPETcomponents = label(PETthresh, structurePET3D) # works out how many 3D components
    
    print("Number of regions > threshold = ", nPETcomponents)
    
    labelvalue = nPETcomponents+2 # defineds a unique label to use for later

    for PETcomponent in range (1,nPETcomponents+1): #for each component of PET image greater than threshold

        print("PETcomponent ", PETcomponent)
        print("BB size = ", np.count_nonzero(BBarrayNewResize == 1))

        n = np.count_nonzero(PETLabelled == PETcomponent)
        print("size of component (n) = ", n)

        # make array with only that component and set value = unique label
        arrayComponent = np.copy(PETLabelled)
        np.putmask(arrayComponent, arrayComponent == PETcomponent, (labelvalue))
        np.putmask(arrayComponent, arrayComponent != (labelvalue), 0)

        m = np.count_nonzero(arrayComponent == (labelvalue))
        print("size of component (m) = ", m)

        # multiply component array with binary BB
        # if component is totally within the BB then all values maintain their label
        # if component is totally outside the BB then all values maintain become 0
        # if component is on edge of BB then only some of the elements will retain their labelled value
        arraynew = np.multiply(arrayComponent,BBarrayNewResize)

        mstar = np.count_nonzero(arraynew == (labelvalue))
        print("mstar = ", mstar)

        if mstar == n:     # i.e. all elements maintained their label and the region is within bowel

            print("************************* INSIDE")

        if mstar == 0:     # i.e. all elements were converted to 0, and the region is outside of bowel
            print("**************************OUTSIDE")

        if mstar > 0 and mstar < n:     # some of the elements were converted to 0, the rest maintained value  
            print("*********************************PERIPHERY**") # i.e. region is on periphery
            np.putmask(arraynew, arraynew > 0, 1)
            BBarrayNewResize = BBarrayNewResize - arraynew
            BBsize = np.count_nonzero(BBarrayNewResize == 1)
            print("New BB size = ", BBsize)

        print("_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _")

    Message_label = Label(window, text = "Peripheral PET Removal - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

# SUV extraction


In [ ]:

def SUVextraction():
    
    SUVinBB = np.multiply(BBarrayNewResize,PETarrayNewSUVbw)
    np.putmask(SUVinBB,SUVinBB == 0, np.nan)
    
    MeanValue = np.nanmean(SUVinBB)
    print("Mean SUV = ", MeanValue)
    MedianValue = np.nanmedian(SUVinBB)
    print("Median SUV = ", MedianValue)
    MaxValue = np.nanmax(SUVinBB)
    print("Max SUV = ", MaxValue)
    NinetyFive_Value = np.nanpercentile(SUVinBB, 95)
    print("95 percentile SUV = ", NinetyFive_Value)
    NinetyNine_Value = np.nanpercentile(SUVinBB, 99)
    print("99 percentile SUV = ", NinetyNine_Value)
    
    SUVinBBflat = np.ndarray.flatten(SUVinBB)
    plt.hist(SUVinBBflat, bins=30, range = (0,3))
    plt.title("Histogram")
    plt.show()

    Message_label = Label(window, text = "SUV extraction - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

# Plotting Commands

In [ ]:
def PlotCTwithLabel():

    global sliceToPlot
    sliceToPlot = int(Slice_label.cget("text"))
    
    print(sliceToPlot)
    fig = plt.figure(figsize=(6, 6))

    im1 = plt.imshow(np.fliplr(np.rot90(CTarray[:,:,sliceToPlot-1],3)),cmap='gray')
    plt.clim(-300,100)

    bar1 = FigureCanvasTkAgg(fig, window)
    bar1.get_tk_widget().grid(column = 0, row = 12)
    
    Message_label = Label(window, text = "Plot CT - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

In [ ]:
def PlotCTandBBwithLabel():

    global sliceToPlot
    global BBpredict
    global BBinfSliceNo
    sliceToPlot = int(Slice_label.cget("text"))
    
    print(sliceToPlot)
    print(sliceToPlot-BBinfSliceNo)

    fig1 = plt.figure(figsize=(6, 6))

    im1 = plt.imshow(np.fliplr(np.rot90(CTarrayNewResize[sliceToPlot-1,:,:],2)),cmap='gray')
    plt.clim(-300,500)

    #im2 = CPredict = plt.contour(np.fliplr(np.rot90(BBarrayNewResizeOrig[sliceToPlot-1,:,:],2)), [1], linestyles = 'solid', colors = ['yellow'])
    im2 = plt.imshow(np.fliplr(np.rot90(BBarrayNewResizeOrig[sliceToPlot-1,:,:],2)),alpha=0.2) # transparency is alpha
    #im2 = plt.colorbar()

    im3 = CPredict = plt.contour(np.fliplr(np.rot90(BBarrayNewResize[sliceToPlot-1,:,:],2)), [1], linestyles = 'solid', colors = ['red'])
    #im3 = plt.imshow(np.fliplr(np.rot90(BBarrayNewResize[sliceToPlot-1,:,:],2)),alpha=0.3) # transparency is alpha
    #im3 = plt.colorbar()

    
    bar1 = FigureCanvasTkAgg(fig1, window)
    bar1.get_tk_widget().grid(column = 0, row = 12)   
    
    fig2 = plt.figure(figsize=(6, 6))
    
    im1 = plt.imshow(np.fliplr(np.rot90(CTarrayNewResize[sliceToPlot-1,:,:],2)),cmap='gray')
    plt.clim(-300,500)
    
    im2 = plt.imshow(np.fliplr(np.rot90(PETarrayNewSUVbw[sliceToPlot-1,:,:],2)),alpha = 0.7)
    plt.clim(0,10)

    im3 = CPredict = plt.contour(np.fliplr(np.rot90(BBarrayNewResize[sliceToPlot-1,:,:],2)), [1], linestyles = 'solid', colors = ['red'])
    #im3 = plt.imshow(np.fliplr(np.rot90(BBarrayNewResize[sliceToPlot-1,:,:],2)),alpha=0.5) # transparency is alpha
    #im3 = plt.colorbar()
    
    
    bar2 = FigureCanvasTkAgg(fig2, window)
    bar2.get_tk_widget().grid(column = 1, row = 12)

    

    
    Message_label = Label(window, text = "Plotting - Done",font=("Arial Bold", 8))
    Message_label.grid(column=0,row=8)

# Make Nifti

In [ ]:
def MakeNifti(): 
    
    # the BB prediction needs to be this shape: (dim0, dim1, slices, 1) for the conversion to Nifti
    
    if dim0 == 512 and dim1 == 512:
        BBpredictToUse = BBpredict
    else:
        BBpredictToUse = BBpredictNon512
        
    BBpredictStrip = BBpredictToUse[0,0,:,:,:]
    BBpredictRot = np.rot90(BBpredictStrip,3,(1,2))
    BBpredictRotTrans = np.transpose(BBpredictRot,(1,2,0))
    #applying threshold of 0.5 to BB prediction
    np.putmask(BBpredictRotTrans, BBpredictRotTrans>=0.5, 1)
    np.putmask(BBpredictRotTrans, BBpredictRotTrans<0.5, 0)
    BBpredictRotTransExp = np.expand_dims(BBpredictRotTrans, axis=3)
    savepath = os.path.join(str(PathDicomCT)+'\BBNiftifromNumpy_BBpredict.nii')
    a = float(RefCT.PixelSpacing[0])*-1
    b = float(RefCT.ImagePositionPatient[0])*-1
    c = float(RefCT.ImagePositionPatient[1])*-1

    # RefCT.SpacingBetweenSlices is what is needed for slice separation
    # but for some header files it doesn't appear.
    # Therefore backup is to calculate difference between slice positions
 
    d = round(b_CT[1] - b_CT[0],2)

    # a is the pixel size
    # b and c are the negative of the first two values in the ImagePositionPatient
    # d is the slice separation
    affine =  np.array([[a, 0, 0, b], [0,a, 0, c], [0,0,d, BBinfSlicePos], [0, 0, 0, 1]]) 

    Nifti = nib.Nifti1Image(BBpredictRotTransExp, affine)
    Nifti.get_data_dtype() == np.dtype(np.int16)
    Nifti.header.get_xyzt_units()
    nib.save(Nifti,savepath)

    # need to make a file called BBname in the CT folder
    # needs to have one line:     0|255 0 0|BowelBag
    # where the last part is the name that the user chooses

    f= open(os.path.join(str(PathDicomCT)+"\BBname.txt"),"w+")
    f.write("0|255 0 0|" + Structure_Name_txt.get())

    newPath = PathDicomCT.replace(os.sep, '/')
    stuff_in_string = "plastimatch convert --referenced-ct {}/ --input-ss-img {}/BBNiftifromNumpy_BBpredict.nii --input-ss-list {}/BBname.txt --output-dicom {}/RTSSdicom".format(newPath,newPath,newPath,newPath)
    print(stuff_in_string)
    Plastimatch_label = Label(window, text = "Plastimatch command has been copied to clipboard",font=("Arial Bold", 8))
    Plastimatch_label.grid(column=1,row=10)
    #Plastimatch_label.text = stuff_in_string

    #window.withdraw()
    window.clipboard_clear()
    window.clipboard_append(stuff_in_string)
    #window.update() # now it stays on the clipboard after the window is closed
    #window.destroy()

    

In [ ]:
from scipy import ndimage, misc

In [ ]:
## using numeric scrollable label operated by the mouse wheel

window = tk.Tk()

def mouse_wheel(event):
    global Slice
    # respond to Linux or Windows wheel event
    if event.num == 5 or event.delta == -120:
        Slice -= 1
    if event.num == 4 or event.delta == 120:
        Slice += 1
    Slice_label['text'] = Slice
    #PlotCTwithLabel() # can add this and the plot will refresh when the mousewheel is activated, 
    #but it is slow to refresh and when closed it wants to plot in the jupyter script so window is slow to close

Slice = 40
CTslices = 200

window.title("BowelBag Prediction Program")
window.geometry("1200x900")

ImportPETCT_label = Label(window, text = "location of PETCT dicom files:",font=("Arial Bold", 10))
ImportPETCT_label.grid(column=0,row=0)

ImportPETCT_txt = Entry(window,width = 90)
ImportPETCT_txt.insert(END,'D:\CNNdata\Bowel_Segmentation\RATHL_Data_Validation_Extra')
ImportPETCT_txt.grid(column=0,row=1)

PatientID_label = Label(window, text = "PatientID:",font=("Arial Bold", 10))
PatientID_label.grid(column=0,row=2)

PatientID_txt = Entry(window,width = 90)
PatientID_txt.insert(END,'_RATHL02_1014')
PatientID_txt.grid(column=0,row=3)


Import_bt = Button(window, text = "Import PETCT", command=lambda:[listofDICOMfiles(),ReadDicoms()])
Import_bt.grid(column = 0, row = 4)

Slice_label = Label(window, text = Slice, font=('courier', 18, 'bold'), width=10)
Slice_label.grid(column = 0, row = 5)

window.bind("<MouseWheel>", mouse_wheel)

PlotCT_bt = Button(window, text = "Plot CT", command=PlotCTwithLabel)
PlotCT_bt.grid(column = 0, row = 6)

InfSlice_label = Label(window, text = "Enter Inf Slice of Bowel:", font=("Arial Bold", 10), width=30)
InfSlice_label.grid(column = 1, row = 0)
InfSlice_txt = Entry(window,width = 10)
InfSlice_txt.grid(column=1,row=1)

SupSlice_label = Label(window, text = "Enter Sup Slice of Bowel:", font=("Arial Bold", 10), width=30)
SupSlice_txt = Entry(window,width = 10)
SupSlice_label.grid(column = 1, row = 2)
SupSlice_txt.grid(column=1,row=3)

ArrayMods_bt = Button(window, text = "ArrayMods", command=lambda:[ArrayMods()])
ArrayMods_bt.grid(column = 1, row = 5)

PredictBowelSpace_bt = Button(window, text = "Load Model and Predict", command=lambda:[loadmodel(), predictBB()])
PredictBowelSpace_bt.grid(column = 1, row = 6)                          
                    
Resampling_bt = Button(window, text = "Resampling", command=lambda:[largestComponent(), BBtoMatchCT(), resampleCT()])
Resampling_bt.grid(column = 1, row = 7)                       
                              
SUVconversion_bt = Button(window, text = "Convert to SUV units", command=lambda:[PETinSUVunits()])
SUVconversion_bt.grid(column = 1, row = 8)                              

PeripheralPETremoval_bt = Button(window, text = "Remove Peripheral PET, then extract SUV data ", command=lambda:[PeripheralPETremoval(),SUVextraction()])
PeripheralPETremoval_bt.grid(column = 1, row = 9)

Fig1_Heading_label = Label(window, text = "Yellow = NN prediction from CT. Red line = Final",font=("Arial Bold", 10))
Fig1_Heading_label.grid(column=0,row=11)

Fig2_Heading_label = Label(window, text = "CT, PET & Final Segmentation",font=("Arial Bold", 10))
Fig2_Heading_label.grid(column=1,row=11)

PlotAll_bt = Button(window, text = "Plot All", command=PlotCTandBBwithLabel)
PlotAll_bt.grid(column = 1, row = 10)


# PredictBB_bt = Button(window, text = "Predict BowelBag", command=clicked_PredictBB)
# PredictBB_bt.grid(column=1,row=4)

# PlotCTandBB_bt = Button(window, text = "Plot CT and BB", command=PlotCTandBBwithLabel)
# PlotCTandBB_bt.grid(column = 1, row = 5)

# Structure_Name_label = Label(window, text = "Structure Name:",font=("Arial Bold", 10))
# Structure_Name_label.grid(column=1,row=7)
# Structure_Name_txt = Entry(window,width = 30)
# Structure_Name_txt.insert(END,'BowelBag')
# Structure_Name_txt.grid(column=1,row=8)

# MakeNifti_bt = Button(window, text = "Make Nifti and Plastimatch String", command=MakeNifti)
# MakeNifti_bt.grid(column = 1, row = 9)

window.lift() # brings window to the front

window.mainloop()

In [ ]:
dir()